### Full Range

策略： 在full range内提供流动性

In [27]:
import sys
sys.path.append("..") 
import pandas as pd
from datetime import datetime
from importlib import reload
from poolData import swapData
from utils import utils
reload(swapData)
reload(utils)

query = swapData.SwapDataQuery()
utils = utils.utils()

In [7]:
# position info
# 选定池子： token0: usdc(decimal: 6), token1: weth(decimal: 18)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
query.query_positions(pool_id=pool_id, block_gte=16859000, limit=10, orderBy="owner")
decimal0 = 6
decimal1 = 18
# 初始头寸的美元价值
initial_position = 10000

# 选定时间
begin = datetime(2023, 3, 19)
end = datetime.now()

pool_id: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 - type: <class 'str'>


In [10]:
# 获取起始时间和结束时间的价格
# 假设本金是1000刀，token0是usdc，数量是500
token0_amount = 5000
# 计算token1数量的时候需要知道价格
# 获取价格
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640")
liquidity_data
price_begin = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][0]))
price_end = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][liquidity_data.shape[0]-1]))
price_begin_real = utils.price2RealPrice(price_begin, decimal0=decimal0, decimal1=decimal1)
price_end_real = utils.price2RealPrice(price_end, decimal0=decimal0, decimal1=decimal1)
# 拿到价格之后才可以计算初始头寸
token1_amount = 5000 * price_begin_real

### 计算该策略的手续费

首先需要计算出头寸的liquidity是多少。

full range策略相当于v2中的头寸，在全价格范围内提供流动性，在v3中就是minTick和maxTick(0,infinity)之间提供流动性。

我在测试网上提供了一个full range的[头寸](https://goerli.etherscan.io/tx/0x199a0a53afbaa7c6b542dc2bdc2766639e9250055032238ebcf204d7375c8aa5), 得到：

tickLower： -887270
tickUpper： 887270


In [18]:
# 计算价格范围
tickLower = -887270
tickUpper = 887270
price_low = utils.tickIndex2Price(tickLower)
price_upper = utils.tickIndex2Price(tickUpper)
print("this is the lower price: ", price_low)
print("this is the upper price: ", price_upper)

this is the lower price:  2.939544628365392e-39
this is the upper price:  3.4018874568203963e+38


In [24]:
# 计算liquidity
l1 = (token0_amount * (price_low ** 0.5) * (price_upper ** 0.5)) / ((price_upper ** 0.5) - (price_low ** 0.5))
l2 = token1_amount / ((price_upper ** 0.5) - (price_low ** 0.5))
l = min(l1, l2)
print("l1:", l1)
print("l2:", l2)
print("l:", l)

l1: 2.710878376267272e-16
l2: 1.5128922860696844e-19
l: 1.5128922860696844e-19


In [13]:
# 获取选定时间期限内的所有swaps
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

this is the cursor: 0x016228aff87377f09e77f120014e8e5ebf14b4f109b9adcf8d25034fce125e39#4692114
this is the cursor: 0x02d9f9b1529f33234c1ff2cbda28122afd13dd2e4833e71789fc6483373a4da8#4645103
this is the cursor: 0x044fba33f804f794d631d21915921e134559286564734d129e3b0c9fb92e4acc#4737144
this is the cursor: 0x05d10812eeaa8b8765c7d9d929c36fdecf975cb66bbc9f8b2103209c6860e6e1#4635383
this is the cursor: 0x074e7d52dfe8335013feb2afee426d4d9bf72ccd2d316a793d0da346d93e36b0#4773378
this is the cursor: 0x08d9c08cda9a348f36944903a14cec7fa120eb30f23a10180d3ce0865fafd649#4622338
this is the cursor: 0x0a51cd41458a2d0a0e9bfc143ae0fff0c81bcb4cf43d43aee54b7af806ed5856#4752694
this is the cursor: 0x0bdcae64222de9182f452b59cc38fddce3ef054ffb87303a1aec7b3f8831b864#4643813
this is the cursor: 0x0d56220f19ccf8eb0f8dd4fdfc22f54803059baedfd8d852b3ba43a2a1520f66#4748513
this is the cursor: 0x0ed8eea0fed4909e2f8e880034a96864645221ea214298bbb3a3e649fade13ee#4615718
this is the cursor: 0x1076d82c9c5e4088aeec5e12cd4f

In [15]:
# 计算fee
# convert str into float
swaps['amount0'] = swaps['amount0'].astype(float)
swaps['amount1'] = swaps['amount1'].astype(float)
swaps['amountUSD'] = swaps['amountUSD'].astype(float)
swaps['sqrtPriceX96'] = swaps['sqrtPriceX96'].astype(float)
swaps['tick'] = swaps['tick'].astype(int)
swaps['logIndex'] = swaps['logIndex'].astype(int)
# transform X86 into normal number
swaps["sqrtPriceX96"] = (swaps["sqrtPriceX96"] / (2**96) ) ** 2
# transform sqrt_p into p
swaps["sqrtPriceX96"] = 1 / (swaps["sqrtPriceX96"] * (10 ** 12))

In [16]:
# 现在来加总然后计算手续费
swaps1 = swaps
# 先把不用的列去掉
swaps1 = swaps1.drop(swaps1.columns[[0, 2, 3, 4, 7, 10]], axis = 1)
swaps1.head(6)

,timestamp,amount0,amount1,sqrtPriceX96,tick
0,1679680919,-19922.526611,11.442382,0.0,201692
1,1680218951,1104.872406,-0.617085,0.0,201422
2,1681392455,-3951.498533,1.973163,0.0,200293
3,1679774651,-12033.180287,7.000000,0.0,201820
4,1681089035,3299.547442,-1.774376,0.0,201044
5,1679404235,-5000.197259,2.750358,0.0,201260


In [29]:
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

In [30]:
liquidity_data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1679158800,31691854789759819627,1871666948661873416199899917784088,1791.851542392230339967547914533666,0.0005580819483878331967604985081167192,201410,2314975469199928457298700227744982,1111162292302232016625192018264273618935405,362517939.1226720089381000671278629,0,0,0,0,372,1798.390894813620679474180985197866,1803.236399827628569085607995221095,1769.953587075568576406795835267567,1791.851542392230339967547914533666
1,1679162400,32915185914729732691,1867893400320714844515651086096830,1799.098710137099104209582500255936,0.0005558338707962253356429190522474716,201369,2315038215475779131181192686558395,1111189202831556094328153678234461295869361,363811618.6075028876192885997227013,0,0,0,0,366,1790.640320160079804025064362917098,1799.108571408040728063168297875355,1786.686354081499257894742581324593,1799.098710137099104209582500255936
2,1679166000,34369179026466248085,1869482743679436927327266292647007,1796.040997899866322473028892703993,0.0005567801632419932350116030494839081,201386,2315126854003039415933596971303595,1111241826769754789231620166343530092470985,363967492.8611761592527996744286012,0,0,0,0,333,1799.099225334904656531075427168004,1809.622901299084675474781496124223,1794.120699231514675114167039962852,1796.040997899866322473028892703993
3,1679169600,34183163223601833767,1868781276195693273308669229878293,1797.389578204559120182790388692096,0.000556362411424970994583196746389777,201379,2315168855482994101747218874079807,1111263525863761589158339866419647898430929,363205815.9317762712257394030005428,0,0,0,0,317,1796.054690522833853305120372567828,1805.57781837591635090288239389413,1792.725482994450340228488201862034,1797.389578204559120182790388692096
4,1679173200,34246952167731560476,1875752556320845172108591732547304,1784.054320332921814271944303424757,0.0005605210495011106585938907612211571,201453,2315193497548537858877306471738285,1111292420021385719852093400706179840724576,360909125.4200933323216227809336806,0,0,0,0,309,1797.199358810234823401563218695319,1797.220201287329445413990798860576,1783.038903387140753714413070004309,1784.054320332921814271944303424757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,1681606800,41636798865354911140,1732128643180098928436239086022382,2092.179202485663651666209180703963,0.0004779705289164169253306741970641501,199860,2346038268696599210433496168993388,1127763700862371094356591121488637197701083,477571150.2894847507413068498250342,0,0,0,0,223,2091.583243996486347312424305007179,2092.298327500000274865477823810146,2075.784547903063805619760673686881,2092.179202485663651666209180703963
681,1681610400,41724563025833799374,1732578802848533141514225021650343,2091.092161200018491048713372288146,0.0004782189989302663534986786503547928,199865,2346044566317075584783921716857144,1127767721107575630127846007565444092087634,477725612.1833312463863818458029293,0,0,0,0,169,2091.966114624023513978051764162945,2092.709185425420739619967026491143,2090.742426770347621380265775639161,2091.092161200018491048713372288146
682,1681614000,38398515815737005928,1733756404590418089572802649097626,2088.252503351387495535238794493957,0.0004788692930548980256266974349508116,199879,2346053937537680078038449467226896,1127774721616050996361553245811001130394769,477773215.7568982857769376527069579,0,0,0,0,182,2091.090899590187621724880645176614,2091.156780111415149634496303173089,2085.919981282628167695180214755831,2088.252503351387495535238794493957
683,1681617600,42457935712613081560,1729886428601702006347382785898618,2097.606327475141858155544496572904,0.0004767338784697915240118188904006587,199834,2346078255994973019672638816314916,1127778780425628529359880692306838410287546,479092674.1179447132512666873922434,0,0,0,0,173,2088.261164803886063450863185442219,2098.444462946398438390743153968

In [32]:
liquidity_data = liquidity_data[["periodStartUnix", "liquidity"]]
liquidity_data = liquidity_data.rename(columns={"periodStartUnix": "timestamp", "liquidity": "liquidity"})
# 先排序
swaps["timestamp"] = swaps["timestamp"].astype(int)
swaps = swaps.sort_values(by="timestamp")
liquidity_data = liquidity_data.sort_values(by="timestamp")
# 找liquidity data只时间最接近的来合并
merged_data = pd.merge_asof(
    swaps,
    liquidity_data[["timestamp", "liquidity"]],
    on="timestamp",
    direction="nearest"
)
merged_data.head(3)

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex,liquidity
0,0x0861d00acafcd74202e82741c031db7417fbca4c45fe...,1679155223,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,-17615.760520,9.835104,17613.789058,0.0,201409,17,31691854789759819627
1,0x4cf93cf259eec962f9f084cc69e3cf15236819fcf3b2...,1679155259,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x94dbb3db4102181186fd127e6f81564d1d4a20a3,729.340475,-0.407206,729.263797,0.0,201419,162,31691854789759819627
2,0x3da2883fd656b389ddce46a794901afc9f0d68ec3c3a...,1679155295,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x3392ef365f255d735a0bd0b83ebca213ba7b4fc4,-256165.425339,143.200000,256297.538930,0.0,201421,13,31691854789759819627


In [33]:
# 获取头寸的liquidity，后续手续费的分成是按照头寸的liquidity占总liquidity的比例来计算的
position_liquidity = l
merged_data['liquidity'] = merged_data['liquidity'].astype(float)
merged_data['ratio'] = float(position_liquidity) / merged_data["liquidity"]
merged_data['amount0'] = merged_data['amount0'].astype(float).apply(lambda x: max(0, x))
merged_data['amount1'] = merged_data['amount1'].astype(float).apply(lambda x: max(0, x))

merged_data["fee0"] = merged_data["amount0"] * merged_data['ratio'] * 0.0005
fee0_sum = merged_data["fee0"].sum()

merged_data["fee1"] = merged_data["amount1"] * merged_data['ratio'] * 0.0005
fee1_sum = merged_data["fee1"].sum()

print(fee0_sum)
print(fee1_sum)

5.8803743662371765e-33
3.140971226804579e-36
